Import Related Library

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re #Regular expression operations¶
import matplotlib.pyplot as plt #plot
import seaborn as sns #statistical data visualization¶
from sklearn.preprocessing import LabelEncoder

View The Dataset

In [ ]:
train = pd.read_csv("../input/train.csv")
train.head(6)

In [ ]:
test = pd.read_csv("../input/test.csv")
test.head(6)

view the statistical properties of the data

In [ ]:
sns.set(style="white", palette="muted", color_codes=True)
f, axes = plt.subplots(figsize=(20,10))
sns.despine(left=True)
d = train['SalePrice']
sns.distplot(d,color='m')
plt.setp(axes, yticks=[])
plt.tight_layout()

Preprocess the features

In [ ]:
train.dtypes

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
test_ID = test['Id']

In [ ]:
train = train.drop('Id',axis=1)
test = test.drop('Id',axis=1)

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.values[:,79]
train = train.drop('SalePrice',axis=1)
data = pd.concat((train, test)).reset_index(drop=True)
#data.drop(['SalePrice'], axis=1, inplace=True)
print("data size is : {}".format(data.shape))

In [ ]:
data.head(6)

In [ ]:
data_na = (data.isnull().sum() / len(data)) * 100
data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :data_na})
missing_data.head(10)

In [ ]:
#Correlation map to see how features are correlated with SalePrice
corrmat = train.corr()
plt.subplots(figsize=(12,9))
sns.heatmap(corrmat, vmax=0.9, square=True)

In [ ]:
#fill the NA with "None"
for na in data.isnull():
    data[na] = data[na].fillna("None")

In [ ]:
data.dtypes

In [ ]:
data.head(6)

In [ ]:
#col = list(train.columns.values)

In [ ]:
int_col = list(data.select_dtypes(include=[np.number]).columns.values)

In [ ]:
col = list(set(list(train.columns.values))-set(int_col))

In [ ]:
print(col)

In [ ]:
for c in col:
    en = LabelEncoder() 
    en.fit(list(data[c].values)) 
    data[c] = en.transform(list(data[c].values))

In [ ]:
data.head(6)

In [ ]:
data.dtypes

Select the best model

Develop a Baseline Neural Network Model

In [ ]:
X_train = data.values[0:1460,0:80]
'''
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(79, input_dim=79, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
'''
print(X_train.shape)
print(type(X_train))
print(y_train.shape)
print(type(y_train))

In [ ]:
X_test = data.values[1460:,0:80]
print(X_test.shape)

In [ ]:
from sklearn.linear_model import LassoLars
reg = LassoLars(alpha=.1)
reg.fit(X_train,y_train)
predictions = reg.predict(X_test)

In [ ]:
i=0
for x in predictions:
    if x<0:
        predictions[i]=-x
    i=i+1
        

In [ ]:
print(predictions.shape)

In [ ]:
sns.set(style="white", palette="muted", color_codes=True)
f, axes = plt.subplots(figsize=(20,10))
sns.despine(left=True)
d = predictions
sns.distplot(d,color='m')
plt.setp(axes, yticks=[])
plt.tight_layout()

In [ ]:
# Generate Submission File 
output = pd.DataFrame({ 'Id': test_ID, 'SalePrice': predictions })
output.to_csv("output.csv", index=False)